In [1]:
# ===== Célula 1: Imports e Configuração de Caminhos =====
import pandas as pd
import numpy as np
from pathlib import Path
import re

# Configuração de exibição do Pandas
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200) # Aumentar para ver mais linhas se necessário
pd.set_option('display.width', 120)    # Ajustar largura para melhor visualização

# --- Configuração de Caminhos ---
project_root = Path('..') # Assume que o notebook está em Analysis/
output_dir   = project_root / 'Output'
analysis_dir = project_root / 'Analysis'

# Caminhos para os arquivos gerados
path_cosif_final    = output_dir / 'df_cosif_final.parquet'
path_ifdata_final   = output_dir / 'df_ifdata_final.parquet'
path_map_prud_geral = output_dir / 'df_mapeamento_prudencial_geral.parquet' # Usaremos o geral
path_banks_csv      = analysis_dir / 'Banks.csv'

print(f"Pasta Output: {output_dir.resolve()}")
print(f"Pasta Analysis: {analysis_dir.resolve()}")
print("-" * 50)

Pasta Output: C:\Users\Enzo\Documents\Programming\b3-analise-bancos\Output
Pasta Analysis: C:\Users\Enzo\Documents\Programming\b3-analise-bancos\Analysis
--------------------------------------------------


In [2]:
# ===== Célula 2: Funções Auxiliares de Limpeza (Replicar ou Importar) =====
def standardize_cnpj_base8(series: pd.Series) -> pd.Series:
    if series.empty: return series
    return series.astype(str).str.replace(r'[^0-9]', '', regex=True).str.zfill(8)

In [3]:
# ===== Célula 3: Carregamento dos DataFrames =====
print("\n--- Carregando DataFrames ---")
df_cosif, df_ifdata, df_map_prud, df_banks = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

try:
    df_cosif = pd.read_parquet(path_cosif_final)
    print(f"df_cosif_final carregado. Shape: {df_cosif.shape}")
except FileNotFoundError: print(f"ERRO: {path_cosif_final} não encontrado.")

try:
    df_ifdata = pd.read_parquet(path_ifdata_final)
    print(f"df_ifdata_final carregado. Shape: {df_ifdata.shape}")
except FileNotFoundError: print(f"ERRO: {path_ifdata_final} não encontrado.")

try:
    df_map_prud = pd.read_parquet(path_map_prud_geral)
    print(f"df_mapeamento_prudencial_geral carregado. Shape: {df_map_prud.shape}")
except FileNotFoundError: print(f"ERRO: {path_map_prud_geral} não encontrado.")

try:
    df_banks = pd.read_csv(path_banks_csv, sep=';', dtype=str)
    if 'CNPJ' in df_banks.columns:
        df_banks['CNPJ_ORIGINAL'] = df_banks['CNPJ']
        df_banks['CNPJ'] = standardize_cnpj_base8(df_banks['CNPJ'])
    print(f"df_banks carregado. Shape: {df_banks.shape}")
except FileNotFoundError: print(f"ERRO: {path_banks_csv} não encontrado.")

if df_cosif.empty or df_ifdata.empty or df_map_prud.empty or df_banks.empty:
    print("\nAVISO: Um ou mais DataFrames essenciais não puderam ser carregados. Verifique os caminhos e a execução do DataDownload.ipynb.")
print("-" * 50)


--- Carregando DataFrames ---
df_cosif_final carregado. Shape: (224756, 11)
df_ifdata_final carregado. Shape: (2907128, 25)
df_mapeamento_prudencial_geral carregado. Shape: (161, 3)
df_banks carregado. Shape: (42, 3)
--------------------------------------------------


In [4]:
# ===== Célula 4: Inspeção Básica - df_cosif =====
if not df_cosif.empty:
    print("\n\n--- Inspeção df_cosif_final ---")
    print("Info:")
    df_cosif.info()
    print("\nNulos:")
    print(df_cosif.isnull().sum().sort_values(ascending=False))
    print("\nHead:")
    display(df_cosif.head(3))
    if 'DATA' in df_cosif.columns: print(f"Período COSIF: {df_cosif['DATA'].min()} a {df_cosif['DATA'].max()}")
    if 'CNPJ' in df_cosif.columns: print(f"CNPJs únicos COSIF: {df_cosif['CNPJ'].nunique()}")
    if 'COD_CONGL' in df_cosif.columns:
        print(f"COD_CONGL únicos COSIF: {df_cosif['COD_CONGL'].nunique(dropna=False)}") # Inclui NA como um único
        print(f"Linhas com COD_CONGL nulo COSIF: {df_cosif['COD_CONGL'].isnull().sum()}")
    print("-" * 50)



--- Inspeção df_cosif_final ---
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224756 entries, 0 to 224755
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   DATA              224756 non-null  int64  
 1   DOCUMENTO         224756 non-null  int64  
 2   CNPJ              224756 non-null  object 
 3   AGENCIA           0 non-null       float64
 4   NOME_INSTITUICAO  224756 non-null  object 
 5   TAXONOMIA         224756 non-null  object 
 6   CONTA_COSIF       224756 non-null  int64  
 7   NOME_CONTA_COSIF  224756 non-null  object 
 8   SALDO_COSIF       224756 non-null  float64
 9   COD_CONGL         119978 non-null  object 
 10  NOME_CONGL        119978 non-null  object 
dtypes: float64(2), int64(3), object(6)
memory usage: 18.9+ MB

Nulos:
AGENCIA             224756
NOME_CONGL          104778
COD_CONGL           104778
CNPJ                     0
DOCUMENTO                0
DATA               

,DATA,DOCUMENTO,CNPJ,AGENCIA,NOME_INSTITUICAO,TAXONOMIA,CONTA_COSIF,NOME_CONTA_COSIF,SALDO_COSIF,COD_CONGL,NOME_CONGL
0,202401,4010,00000000,NaN,BCO DO BRASIL S.A.,BANCO DO BRASIL - BANCO MULTIPLO,10000007,ATIVO REALIZÁVEL,2.122392e+12,C0080329,BB - PRUDENCIAL
1,202401,4010,00000000,NaN,BCO DO BRASIL S.A.,BANCO DO BRASIL - BANCO MULTIPLO,11000006,DISPONIBILIDADES,1.633250e+10,C0080329,BB - PRUDENCIAL
2,202401,4010,00000000,NaN,BCO DO BRASIL S.A.,BANCO DO BRASIL - BANCO MULTIPLO,11100009,Caixa,1.305472e+10,C0080329,BB - PRUDENCIAL


Período COSIF: 202401 a 202412
CNPJs únicos COSIF: 175
COD_CONGL únicos COSIF: 82
Linhas com COD_CONGL nulo COSIF: 104778
--------------------------------------------------


In [5]:
# ===== Célula 5: Inspeção Básica - df_ifdata =====
if not df_ifdata.empty:
    print("\n\n--- Inspeção df_ifdata_final ---")
    print("Info:")
    df_ifdata.info()
    print("\nNulos (top 10 e colunas chave):")
    nulos_ifdata = df_ifdata.isnull().sum().sort_values(ascending=False)
    print(nulos_ifdata.head(10))
    cols_chave_if = ['COD_CONGL', 'DATA', 'NOME_CONTA_IF', 'SALDO_IF', 'NOME_INSTITUICAO_IF', 'CNPJ_LIDER_BASE8_IF']
    print("Nulos em colunas chave IFDATA:")
    for col in cols_chave_if:
        if col in nulos_ifdata:
            print(f"  {col}: {nulos_ifdata[col]}")
    print("\nHead:")
    display(df_ifdata.head(3))
    if 'DATA' in df_ifdata.columns: print(f"Período IFDATA: {df_ifdata['DATA'].min()} a {df_ifdata['DATA'].max()}")
    if 'COD_CONGL' in df_ifdata.columns: print(f"COD_CONGL únicos IFDATA: {df_ifdata['COD_CONGL'].nunique()}")
    if 'CNPJ_LIDER_BASE8_IF' in df_ifdata.columns: print(f"CNPJs Líder (base8) únicos IFDATA: {df_ifdata['CNPJ_LIDER_BASE8_IF'].nunique(dropna=False)}")
    print("-" * 50)



--- Inspeção df_ifdata_final ---
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2907128 entries, 0 to 2907127
Data columns (total 25 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   TipoInstituicao           Int64  
 1   COD_CONGL                 object 
 2   DATA                      Int64  
 3   NomeRelatorio             object 
 4   NumeroRelatorio           Int64  
 5   Grupo                     object 
 6   CONTA_IF                  Int64  
 7   NOME_CONTA_IF             object 
 8   DescricaoColuna           object 
 9   SALDO_IF                  float64
 10  DATA_ULTIMO_CADASTRO_IF   float64
 11  NOME_INSTITUICAO_IF       object 
 12  CNPJ_LIDER_IF14           float64
 13  TCB_IF                    object 
 14  TD_IF                     object 
 15  TC_IF                     float64
 16  SEGMENTO_TB_IF            object 
 17  ATIVIDADE_IF              object 
 18  UF_IF                     object 
 19  MUNICIPIO_IF          

,TipoInstituicao,COD_CONGL,DATA,NomeRelatorio,NumeroRelatorio,Grupo,CONTA_IF,NOME_CONTA_IF,DescricaoColuna,SALDO_IF,DATA_ULTIMO_CADASTRO_IF,NOME_INSTITUICAO_IF,CNPJ_LIDER_IF14,TCB_IF,TD_IF,TC_IF,SEGMENTO_TB_IF,ATIVIDADE_IF,UF_IF,MUNICIPIO_IF,SR_IF,COD_CONGL_FINANCEIRO_IF,DATA_INICIO_ATIVIDADE_IF,SITUACAO_IF,CNPJ_LIDER_BASE8_IF
0,1,69136075,202403,Resumo,1,nan,78186,Patrimônio Líquido,[60000002] + [70000009]+ [80000006],3641277.56,NaN,None,NaN,None,None,NaN,None,None,None,None,None,None,NaN,None,00000000
1,1,69136075,202403,Resumo,1,nan,78187,Lucro Líquido,[70000009]+[80000006]-[81956001],39474.15,NaN,None,NaN,None,None,NaN,None,None,None,None,None,None,NaN,None,00000000
2,1,69136075,202403,Resumo,1,nan,79649,Patrimônio de Referência para Comparação com o...,Montante de capital regulatório formado pela s...,3627180.43,NaN,None,NaN,None,None,NaN,None,None,None,None,None,None,NaN,None,00000000


Período IFDATA: 202403 a 202412
COD_CONGL únicos IFDATA: 2802
CNPJs Líder (base8) únicos IFDATA: 610
--------------------------------------------------


In [6]:
# ===== Célula 6: Análise do Mapeamento Prudencial e df_banks =====
if not df_map_prud.empty:
    print("\n\n--- Inspeção df_mapeamento_prudencial_geral ---")
    df_map_prud.info()
    print(f"Mapeamento: CNPJs únicos: {df_map_prud['CNPJ'].nunique()}, COD_CONGL únicos: {df_map_prud['COD_CONGL'].nunique()}")
    display(df_map_prud.head(3))
    print("-" * 50)

if not df_banks.empty:
    print("\n\n--- Inspeção df_banks ---")
    display(df_banks.head(3))
    print(f"Bancos de interesse: {df_banks['CNPJ'].nunique()}")
    print("-" * 50)



--- Inspeção df_mapeamento_prudencial_geral ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CNPJ        161 non-null    object
 1   COD_CONGL   161 non-null    object
 2   NOME_CONGL  161 non-null    object
dtypes: object(3)
memory usage: 3.9+ KB
Mapeamento: CNPJs únicos: 161, COD_CONGL únicos: 160


,CNPJ,COD_CONGL,NOME_CONGL
0,00000000,C0080329,BB - PRUDENCIAL
1,00000208,C0080288,BRB - PRUDENCIAL
2,00360305,C0080738,CAIXA ECONÔMICA FEDERAL - PRUDENCIAL


--------------------------------------------------


--- Inspeção df_banks ---


,NOME,CNPJ,CNPJ_ORIGINAL
0,ABC,28195667,28195667
1,Master,33923798,33923798
2,VOTORANTIM,59588111,59588111


Bancos de interesse: 42
--------------------------------------------------


In [7]:
# ===== Célula 7: Análise de Cobertura dos Bancos de Interesse =====
if not df_banks.empty:
    print("\n\n--- Cobertura dos Bancos de Interesse ---")
    cnpjs_interesse = df_banks['CNPJ'].tolist()

    # Cobertura no COSIF
    if not df_cosif.empty:
        bancos_no_cosif = df_banks[df_banks['CNPJ'].isin(df_cosif['CNPJ'].unique())]
        print(f"{len(bancos_no_cosif)} de {len(df_banks)} bancos de interesse encontrados em df_cosif.")
        bancos_ausentes_cosif = df_banks[~df_banks['CNPJ'].isin(df_cosif['CNPJ'].unique())]
        if not bancos_ausentes_cosif.empty:
            print("Bancos de interesse AUSENTES no COSIF:")
            display(bancos_ausentes_cosif[['NOME', 'CNPJ']])
        
        # Desses no COSIF, quantos têm COD_CONGL?
        bancos_cosif_com_cod_congl = pd.merge(bancos_no_cosif, df_cosif[['CNPJ', 'COD_CONGL']].drop_duplicates(), on='CNPJ', how='left')
        print(f"Dos {len(bancos_no_cosif)} bancos no COSIF, {bancos_cosif_com_cod_congl['COD_CONGL'].notnull().sum()} têm um COD_CONGL mapeado.")
    else:
        print("df_cosif vazio, não é possível verificar cobertura.")

    # Cobertura no IFDATA (via mapeamento e via CNPJ_LIDER)
    if not df_ifdata.empty:
        # Via mapeamento CNPJ -> COD_CONGL -> IFDATA
        if not df_map_prud.empty:
            mapeamento_interesse = pd.merge(df_banks[['CNPJ', 'NOME']], df_map_prud, on='CNPJ', how='left')
            mapeamento_interesse_com_ifdata = mapeamento_interesse[mapeamento_interesse['COD_CONGL'].isin(df_ifdata['COD_CONGL'].unique())]
            print(f"{mapeamento_interesse_com_ifdata['CNPJ'].nunique()} de {len(df_banks)} bancos de interesse têm COD_CONGL com dados no IFDATA.")
        
        # Via CNPJ_LIDER_BASE8_IF
        if 'CNPJ_LIDER_BASE8_IF' in df_ifdata.columns:
            bancos_lideres_ifdata = df_banks[df_banks['CNPJ'].isin(df_ifdata['CNPJ_LIDER_BASE8_IF'].unique())]
            print(f"{len(bancos_lideres_ifdata)} de {len(df_banks)} bancos de interesse são CNPJ_LIDER_BASE8_IF no IFDATA.")
    else:
        print("df_ifdata vazio, não é possível verificar cobertura.")
    print("-" * 50)
else:
    print("df_banks vazio, pulando análise de cobertura.")



--- Cobertura dos Bancos de Interesse ---
35 de 42 bancos de interesse encontrados em df_cosif.
Bancos de interesse AUSENTES no COSIF:


,NOME,CNPJ
23,Nubank,30680829
28,Qista SA,36583700
32,Caruana,09313766
33,DM FINANCEIRA,91669747
37,Agrolend,43774196
40,Plantae,35551187
41,Mercado Crédito CFI,37679449


Dos 35 bancos no COSIF, 28 têm um COD_CONGL mapeado.
30 de 42 bancos de interesse têm COD_CONGL com dados no IFDATA.
0 de 42 bancos de interesse são CNPJ_LIDER_BASE8_IF no IFDATA.
--------------------------------------------------


In [8]:
# ===== Célula 8: Busca Detalhada para Banco Inter (Exemplo) =====
print("\n\n--- Busca Detalhada: Banco Inter (00416968) para 202412 ---")
cnpj_alvo = '00416968'
data_alvo = 202412 # Certifique-se que esta data existe nos seus dados

banco_alvo_info = df_banks[df_banks['CNPJ'] == cnpj_alvo]
if not banco_alvo_info.empty:
    print(f"Informações do Banco Alvo (de df_banks):")
    display(banco_alvo_info)
else:
    print(f"CNPJ {cnpj_alvo} não encontrado em df_banks.")

# 1. Buscar dados COSIF
if not df_cosif.empty:
    print(f"\n--- Dados COSIF para CNPJ {cnpj_alvo}, Data {data_alvo} ---")
    df_cosif_banco = df_cosif[(df_cosif['CNPJ'] == cnpj_alvo) & (df_cosif['DATA'] == data_alvo)]
    if not df_cosif_banco.empty:
        print(f"Encontradas {len(df_cosif_banco)} linhas no COSIF.")
        # Mostrar algumas contas COSIF relevantes (exemplo)
        contas_cosif_interesse = ['ATIVO TOTAL', 'PATRIMONIO LIQUIDO', 'LUCRO LIQUIDO DO SEMESTRE', 'OPERACOES DE CREDITO']
        # Normalizar nomes para busca mais flexível
        df_cosif_banco_display = df_cosif_banco[
            df_cosif_banco['NOME_CONTA_COSIF'].str.upper().str.contains('|'.join(contas_cosif_interesse), na=False, case=False)
        ]
        if not df_cosif_banco_display.empty:
            display(df_cosif_banco_display[['NOME_CONTA_COSIF', 'SALDO_COSIF', 'COD_CONGL', 'NOME_CONGL']])
        else:
            print(f"Nenhuma das contas COSIF de exemplo ({contas_cosif_interesse}) encontrada para {cnpj_alvo} em {data_alvo}.")
            print("Contas COSIF disponíveis para este banco/data (até 10):")
            display(df_cosif_banco[['NOME_CONTA_COSIF', 'SALDO_COSIF']].head(10))
        
        cod_congl_do_cosif = df_cosif_banco['COD_CONGL'].iloc[0] if pd.notna(df_cosif_banco['COD_CONGL'].iloc[0]) else None
        print(f"COD_CONGL associado no COSIF: {cod_congl_do_cosif}")
    else:
        print(f"Nenhum dado COSIF encontrado para CNPJ {cnpj_alvo} na data {data_alvo}.")
        cod_congl_do_cosif = None # Inicializa para o próximo passo
else:
    print("df_cosif vazio.")
    cod_congl_do_cosif = None

# 2. Buscar dados IFDATA
# Primeiro, tentar obter COD_CONGL do mapeamento prudencial se não veio do COSIF
cod_congl_para_ifdata = cod_congl_do_cosif
if not cod_congl_para_ifdata and not df_map_prud.empty:
    map_entry = df_map_prud[df_map_prud['CNPJ'] == cnpj_alvo] # Pega o COD_CONGL mais estável
    if not map_entry.empty:
        cod_congl_para_ifdata = map_entry['COD_CONGL'].iloc[0]
        print(f"COD_CONGL obtido do mapeamento geral: {cod_congl_para_ifdata}")

if cod_congl_para_ifdata and not df_ifdata.empty:
    print(f"\n--- Dados IFDATA para COD_CONGL {cod_congl_para_ifdata}, Data {data_alvo} ---")
    # Nota: IFDATA Valores é trimestral. 202412 é um fim de trimestre.
    df_ifdata_banco = df_ifdata[(df_ifdata['COD_CONGL'] == cod_congl_para_ifdata) & (df_ifdata['DATA'] == data_alvo)]
    if not df_ifdata_banco.empty:
        print(f"Encontradas {len(df_ifdata_banco)} linhas no IFDATA.")
        # Mostrar algumas métricas IFDATA relevantes (exemplo)
        # Verifique os nomes exatos no seu 'dicionario_contas_ifdata.xlsx'
        metricas_ifdata_interesse = ['Índice de Basileia', 'RWA', 'Patrimonio de Referencia', 'Resultado Liquido'] # Use nomes exatos!
        
        df_ifdata_banco_display = df_ifdata_banco[
             df_ifdata_banco['NOME_CONTA_IF'].str.upper().str.contains('|'.join(metricas_ifdata_interesse).upper(), na=False, case=False)
        ]
        if not df_ifdata_banco_display.empty:
            display(df_ifdata_banco_display[['NOME_CONTA_IF', 'SALDO_IF', 'NOME_INSTITUICAO_IF', 'CNPJ_LIDER_BASE8_IF']])
        else:
            print(f"Nenhuma das métricas IFDATA de exemplo ({metricas_ifdata_interesse}) encontrada para {cod_congl_para_ifdata} em {data_alvo}.")
            print("Métricas IFDATA disponíveis para este COD_CONGL/data (até 10):")
            display(df_ifdata_banco[['NOME_CONTA_IF', 'SALDO_IF']].head(10))
            
        # Mostrar informações cadastrais do IFDATA para este conglomerado (se houver)
        info_cadastral_if = df_ifdata_banco[
            ['NOME_INSTITUICAO_IF', 'CNPJ_LIDER_BASE8_IF', 'TCB_IF', 'SITUACAO_IF', 'DATA_ULTIMO_CADASTRO_IF']
        ].drop_duplicates().head(1) # Pega a primeira entrada única
        if not info_cadastral_if.empty and info_cadastral_if.notna().any().any() :
            print("\nInformações Cadastrais do IFDATA (mais recentes para este COD_CONGL):")
            display(info_cadastral_if)

    else:
        print(f"Nenhum dado IFDATA encontrado para COD_CONGL {cod_congl_para_ifdata} na data {data_alvo}.")
elif not df_ifdata.empty:
    print(f"\nNão foi possível determinar o COD_CONGL para o CNPJ {cnpj_alvo} para buscar dados IFDATA.")
else:
    print("df_ifdata vazio.")
print("-" * 50)



--- Busca Detalhada: Banco Inter (00416968) para 202412 ---
Informações do Banco Alvo (de df_banks):


,NOME,CNPJ,CNPJ_ORIGINAL
12,INTER,00416968,00416968



--- Dados COSIF para CNPJ 00416968, Data 202412 ---
Encontradas 227 linhas no COSIF.


,NOME_CONTA_COSIF,SALDO_COSIF,COD_CONGL,NOME_CONGL
195810,PATRIMONIO LIQUIDO,6.867869e+09,C0080996,INTER - PRUDENCIAL
212420,PATRIMONIO LIQUIDO,7.256024e+09,C0080996,INTER - PRUDENCIAL


COD_CONGL associado no COSIF: C0080996

--- Dados IFDATA para COD_CONGL C0080996, Data 202412 ---
Encontradas 105 linhas no IFDATA.


,NOME_CONTA_IF,SALDO_IF,NOME_INSTITUICAO_IF,CNPJ_LIDER_BASE8_IF
2194705,Patrimônio de Referência para Comparação com o...,5.262321e+09,INTER PAG INSTITUIÇÃO DE PAGAMENTO S.A.,04169680
2194706,Índice de Basileia,1.518543e-01,INTER PAG INSTITUIÇÃO DE PAGAMENTO S.A.,04169680
2279653,Capital Principal para Comparação com RWA (a),5.262321e+09,INTER PAG INSTITUIÇÃO DE PAGAMENTO S.A.,04169680
2279655,Patrimônio de Referência Nível I para Comparaç...,5.262321e+09,INTER PAG INSTITUIÇÃO DE PAGAMENTO S.A.,04169680
2279662,Patrimônio de Referência para Comparação com o...,5.262321e+09,INTER PAG INSTITUIÇÃO DE PAGAMENTO S.A.,04169680
2279663,RWA para Risco de Crédito (f),2.705337e+10,INTER PAG INSTITUIÇÃO DE PAGAMENTO S.A.,04169680
2279667,RWAcam (g1),1.278662e+08,INTER PAG INSTITUIÇÃO DE PAGAMENTO S.A.,04169680
2279668,RWAcom (g2),7.408519e+07,INTER PAG INSTITUIÇÃO DE PAGAMENTO S.A.,04169680
2279672,RWAjur (g3),4.444247e+08,INTER PAG INSTITUIÇÃO DE PAGAMENTO S.A.,04169680
2279673,RWAacs (g4),3.004574e+06,INTER PAG INSTITUIÇÃO DE PAGAMENTO S.A.,04169680



Informações Cadastrais do IFDATA (mais recentes para este COD_CONGL):


,NOME_INSTITUICAO_IF,CNPJ_LIDER_BASE8_IF,TCB_IF,SITUACAO_IF,DATA_ULTIMO_CADASTRO_IF
2191252,INTER PAG INSTITUIÇÃO DE PAGAMENTO S.A.,04169680,N4,A,202412.0


--------------------------------------------------


In [9]:
# ===== Célula 9: Conclusões Preliminares e Próximos Passos =====
print("""
\n\n--- Conclusões Preliminares e Próximos Passos ---
1. Avalie os Nulos:
   - `df_cosif['COD_CONGL']`: Os nulos aqui indicam CNPJs do COSIF Individual sem mapeamento no Prudencial. São instituições independentes?
   - `df_ifdata`: Nulos em colunas como `NOME_INSTITUICAO_IF`, `CNPJ_LIDER_IF14` indicam que muitos `COD_CONGL` com dados de VALORES não têm entrada rica no CADASTRO do IFDATA. Isso é uma limitação da fonte.

2. Cobertura dos Bancos de Interesse:
   - Quais bancos do seu `df_banks` não foram encontrados no COSIF? Por quê?
   - Para os encontrados, quantos puderam ser ligados a dados IFDATA via `COD_CONGL`?
   - Algum dos seus bancos de interesse é um `CNPJ_LIDER_BASE8_IF`? (Seu output anterior indicou 0).

3. Exemplo do Banco Inter:
   - Conseguiu puxar dados COSIF e IFDATA? Se não, por quê? (Nome da conta/métrica incorreto, data inexistente, falha no mapeamento CNPJ->COD_CONGL).
   - Os valores fazem sentido?

4. Qualidade dos Dados:
   - Os nomes de contas/métricas estão limpos?
   - Os saldos parecem estar na escala correta?

Próximos Passos:
- Aprofundar a investigação para bancos/métricas específicas de interesse.
- Se a cobertura para `df_banks` for baixa, investigar os motivos para cada banco ausente.
- Adaptar a célula 8 para outros bancos e datas.
- Começar a construir visualizações ou tabelas de resumo com os indicadores chave.
- Se a "perda de linhas" no IFDATA (de ~3.28M para ~2.9M) ainda for uma preocupação,
  revisitar o encoding/leitura dos CSVs `IfDataValores` no `DataDownload.ipynb`.
  (Tentar `encoding='latin1'` para `IfDataValores` e ver se o `shape` de `df_if_valores_raw` aumenta).
""")




--- Conclusões Preliminares e Próximos Passos ---
1. Avalie os Nulos:
   - `df_cosif['COD_CONGL']`: Os nulos aqui indicam CNPJs do COSIF Individual sem mapeamento no Prudencial. São instituições independentes?
   - `df_ifdata`: Nulos em colunas como `NOME_INSTITUICAO_IF`, `CNPJ_LIDER_IF14` indicam que muitos `COD_CONGL` com dados de VALORES não têm entrada rica no CADASTRO do IFDATA. Isso é uma limitação da fonte.

2. Cobertura dos Bancos de Interesse:
   - Quais bancos do seu `df_banks` não foram encontrados no COSIF? Por quê?
   - Para os encontrados, quantos puderam ser ligados a dados IFDATA via `COD_CONGL`?
   - Algum dos seus bancos de interesse é um `CNPJ_LIDER_BASE8_IF`? (Seu output anterior indicou 0).

3. Exemplo do Banco Inter:
   - Conseguiu puxar dados COSIF e IFDATA? Se não, por quê? (Nome da conta/métrica incorreto, data inexistente, falha no mapeamento CNPJ->COD_CONGL).
   - Os valores fazem sentido?

4. Qualidade dos Dados:
   - Os nomes de contas/métricas estão lim